#  Data Acquisition & Data Analyse Reporting

### Introduction

Ce notebook constitue le rapport des modules Data Acquisition et Dana Analyse. Ce rapport fait donc office d'explication des modules python. Il permet nottament d'expliquer 
- le role de chaque module,
- les fonctions et les concepts implementés

NB : Le code présenté ne constitue pas le code source des module. C'est un code simplifié qui facilite la compréhension des systèmes. Les fonctionnalitées non codés dans ce rapport seront tout de même expliquées et documéntées

Ce notebook est à titre d'explication. Seul les fonctionnalitées essentielles sont présentées, le but étant de à des de fin de compréhension. Les autres fonctionnalitées seront tout de même expliquées et documéntées

##### download the requierement 

Elasticsearch6 package is used but you can also use elastiscsearch (that will be work)

In [1]:
"""
!pip install elasticsearch6
!pip install pandas
"""

'\n!pip install elasticsearch6\n!pip install pandas\n'

# Data Acquisition

#### Setup variables

In [2]:
import os

In [3]:
host='localhost'
port = 9200
path_data= f'{os.getcwd()}/data'

create a directory for the data that will be save

In [4]:
def create_directory(path_directory):
    if not os.path.exists(path_directory):
        os.mkdir(path_directory)
create_directory(path_data)

In [5]:
path_data

'/home/eguimard/PycharmProjects/DataStudy/TestDev/Notebook/data'

### Connecting To Cluster

Connect To ElasticSearch Cluster with python-api

In [6]:
from elasticsearch6 import Elasticsearch

In [7]:
client = Elasticsearch(f'{host}:{port}')

#### Index  for the search :

In [8]:
index= "events-20220502"

### Make a query  on the Elastic Cluster

here it's a json elastic query wrote in dict python. With this query, we recover all the data where category == 'availability' in the index "events-20220502". 
Here we chose perimeter availability for the report, but it's pretty much the same process for the others perimeters

In [9]:
perimeter ="availability"

In [10]:
VM ="VM1"

In [11]:
query_in_dict ={
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "category": perimeter
          }
        },
        {
          "match": {
            "hostname": VM
          }
        }
      ],
      "must_not": [],
      "should": []
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [],
  "aggs": {}
}

Here we run the query on the cluster. <br>The output correpond to the return of the query. We will name this raw_data

In [12]:
raw_data = client.search(index=index, body=query_in_dict)
raw_data

{'took': 182,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': 710355,
  'max_score': 1.891559,
  'hits': [{'_index': 'events-20220502',
    '_type': '_doc',
    '_id': '7gWVkoABdRhNzRXUqF3b',
    '_score': 1.891559,
    '_source': {'summary': 'Memory statistics',
     'category': 'availability',
     'client_id': 'airliquide',
     'client_key': '69995ba006ba4ec3a86e4730343be794',
     'timestamp': '2021-09-30T03:28:52.147Z',
     'hostname': 'VM1',
     'source': 'metricbeat',
     'tags': ['metricbeat.system.memory'],
     'details': {'metricset': {'name': 'memory', 'period': 15000},
      'event': {'duration': 1951453,
       'module': 'system',
       'dataset': 'system.memory'},
      'service': {'type': 'system'},
      'ecs': {'version': '1.5.0'},
      'agent': {'id': '69c94e43-b718-4f50-8636-ef344572580d',
       'type': 'metricbeat',
       'version': '7.8.1',
       'hostname': 'VM1',
       'name': 'VM1',
       

### Extracting

Here we will exctract the relevant data drom raw_data for futures analysis. The relevant data are twice : 
- the system data accesible at raw_data['hits']['hits'][n]['_source']['details']['system']
- a list of wanted feature that we can define. Features are accesible at raw_data['hits']['hits'][n]['_source']

Here we define a list of wanted feature to recover. In our cas we just need 'utctimestamp' for the data visualisation

In [13]:
list_wanted_features = ['utctimestamp']

the main function of the data extracting :

In [14]:
def exctract_relevant_data_from_raw_data(raw_data, list_wanted_features):
    
    dict_of_list_dict_data = dict()
    
    for results in raw_data['hits']['hits']:
        
        # extract system data recursively and append to dict_final
        system_data = results['_source']['details']['system']
        system_name = list(system_data.keys())[0]
        
        dict_extracted_data = develop_data_recursively_for_extraction(system_data[system_name])
        
        # wanted_features extraction
        dict_extracted_data = extract_wanted_features(results['_source'], list_wanted_features, dict_extracted_data)

        
        # append dict_extracted_data to the corresponding list
        if system_name not in dict_of_list_dict_data:
            dict_of_list_dict_data[system_name] = list()
        dict_of_list_dict_data[system_name].append(dict_extracted_data)
        
    return dict_of_list_dict_data
    

this function  goes back recursively through dict of dict and create feature name foreach value found

In [15]:
def develop_data_recursively_for_extraction(current_dict, name_feature=''):

    dict_final = dict()

    if type(current_dict) is dict:
        for key, new_obj in current_dict.items():
            dict_final.update(develop_data_recursively_for_extraction(new_obj, f'{name_feature}{key}_'))
            # update can delete field. in theory, we don't have twice the same field
    else:
        dict_final[name_feature[:-1]] = current_dict

    return dict_final

this function allows to extract feature from a list of name : 

In [16]:
def extract_wanted_features(current_data, list_wanted_features, dict_extracted_data):

    for wanted_feature in list_wanted_features:
        dict_extracted_data[wanted_feature] = current_data[wanted_feature]

    return dict_extracted_data

In [17]:
dict_list_data = exctract_relevant_data_from_raw_data(raw_data, list_wanted_features)

In output we have a dictionnary of list. We will convert these lists to Dataframe : 

In [18]:
import pandas as pd

In [19]:
def transform_list_data_to_dataframe(dict_of_list_data):
    dict_df = dict()

    for key, list_data in dict_of_list_data.items():
        dict_df[key] = pd.DataFrame(list_data)

    return dict_df

In [20]:
dict_dataFrame = transform_list_data_to_dataframe(dict_list_data)

The name of the dataframes created are the keys of dictionnary : 

In [21]:
dict_dataFrame.keys()

dict_keys(['memory', 'fsstat', 'cpu', 'network'])

Each DataFrame is related to a system perimeter (in the availability case) like memory, cpu, network...

Finnaly we Save the dict dataframe to the path data define at the top of this notebook. The dataframes stores will be use in the Notebook Data Analise : 

In [23]:
def store_dict_df_into_directory(dict_df, save_path_directory, extension):

    for name_df in dict_df.keys():
        try:
            dict_df[name_df].to_csv(f'{save_path_directory}/{name_df}.{extension}', index=False)
        except Exception as err:  # precisez l'exception
            return str(err)

In [24]:
store_dict_df_into_directory(dict_dataFrame, path_data, extension='csv')

In [25]:
path_data

'/home/eguimard/PycharmProjects/DataStudy/TestDev/Notebook/data'